# Pokemon TCG API has a [Python SDK](https://github.com/PokemonTCG/pokemon-tcg-sdk-python)
By installing this SDK, I can speed up the development process. This eliminates a lot of work interfacing nicely with the API.

### Configure REST Client with API Key


In [1]:
from pokemontcgsdk import RestClient
import yaml as yaml

with open('../sensitive_config/api_config.yaml','r') as f:
    config = yaml.safe_load(f)

RestClient.configure(config['api_key'])

### Test Queries

In [5]:
from pokemontcgsdk import Card
from pokemontcgsdk import Set
from pokemontcgsdk import Type
from pokemontcgsdk import Supertype
from pokemontcgsdk import Subtype
from pokemontcgsdk import Rarity

### Testing Static Dimension Tables

In [ ]:
supertypes = Supertype.all()
supertypes

In [9]:
subtypes = Subtype.all()
subtypes[0:4]

['ACE SPEC', 'Ancient', 'BREAK', 'Baby']

In [10]:
types = Type.all()
types[0:4]

['Colorless', 'Darkness', 'Dragon', 'Fairy']

In [13]:
rarities = Rarity.all()
rarities[-4:]

['Special Illustration Rare',
 'Trainer Gallery Rare Holo',
 'Ultra Rare',
 'Uncommon']

In [27]:
import json
from pathlib import Path

data_path = Path().cwd().parent / 'data'
raw_data_path = data_path / 'raw'

In [29]:
with open(raw_data_path / 'rarities.json', 'w') as f:
    json.dump(rarities, f, indent=4)

with open(raw_data_path / 'types.json', 'w') as f:
    json.dump(types, f, indent=4)

with open(raw_data_path / 'subtypes.json', 'w') as f:
    json.dump(subtypes, f, indent=4)

with open(raw_data_path / 'supertypes.json', 'w') as f:
    json.dump(supertypes, f, indent=4)

### Testing Set API

In [ ]:
import dataclasses as dc

In [32]:
sets_catalog = Set.all()
# Convert list of Dataclass objects to list of dictionaries
sets_catalog_as_dict: list[dict] = [dc.asdict(card_set) for card_set in sets_catalog]
sets_catalog_as_dict[0]

{'id': 'base1',
 'images': {'symbol': 'https://images.pokemontcg.io/base1/symbol.png',
  'logo': 'https://images.pokemontcg.io/base1/logo.png'},
 'legalities': {'unlimited': 'Legal', 'expanded': None, 'standard': None},
 'name': 'Base',
 'printedTotal': 102,
 'ptcgoCode': 'BS',
 'releaseDate': '1999/01/09',
 'series': 'Base',
 'total': 102,
 'updatedAt': '2022/10/10 15:12:00'}

In [33]:
with open(raw_data_path / 'sets_catalog.json', 'w') as f:
    json.dump(sets_catalog_as_dict, f, indent=4)

TypeError: asdict() should be called on dataclass instances